# Investigation of an Arrhenius relationship

In addition to being able to determine the mean-squared displacement and diffusion coefficient from a given simulation, `kinisi` also includes [tools](./arrhenius.html) to investigate Arrhenius relationships. 
In this tutorial, we will look at how we can take advantage of these tools to study short, approximately 50 ps, simulations of lithium lanthanum zirconium oxide (LLZO).

```{note}
The warnings that are being ignored are related to the parsing of the files by `MDAnalysis` and lead to unnecessary print out to the screen that we want to avoid in the web documentation.
```

In [ ]:
import numpy as np
import MDAnalysis as mda
import matplotlib.pyplot as plt
from kinisi.analyze import DiffusionAnalyzer
from kinisi.arrhenius import StandardArrhenius
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

To investigate this we will loop through a series of four temperatures and append each diffusion coefficient to a list. 

In [ ]:
temperatures = np.array([500, 600, 700, 800])
D = []

To read these simulations we will use [`MDAnalysis`]((https://userguide.mdanalysis.org/stable/index.html).
The parser, bootstrap, and diffusion parameters are all defined for all simulations, here we only consider the diffusive regime to begin after 5 ps.

In [ ]:
p_params = {'specie': 'LI',
            'time_step': 5.079648e-4,
            'step_skip': 100,
            'min_dt': 0.001,
            'progress': False}
b_params = {'progress': False}
d_params = {'dt_skip': 5,
            'progress': False}

File parsing and diffusion determination is then performed in a loop here. 

In [ ]:
for t in temperatures:
    u = mda.Universe(f'_static/traj_{t}.gro', f'_static/traj_{t}.xtc')
    d = DiffusionAnalyzer.from_universe(u, p_params, bootstrap_params=b_params)
    d.diffusion(d_params)
    D.append(d.D)

The list of diffusion coefficient objects (which are `uravu.distribution.Distribution` type objects) and array of temperatures can then be passed to the `kinisi.arrhenius.StandardArrhenius` class. 

In [ ]:
s = StandardArrhenius(temperatures, D)

Having created the object, we can determine the maximum likelihood values for the parameters of activation energy and the preexponential factor. 

In [ ]:
s.max_likelihood('diff_evo')

After determining the maximum likelihood values, we can use Markov chain Monte Carlo (MCMC) to sample the probability distributions for these. 

In [ ]:
s.mcmc(progress=False)

We can then visualise the probability distributions for the parameters as histograms. 

In [ ]:
fig, ax = plt.subplots(1, 2)
ax[0].hist(s.activation_energy.samples)
ax[0].set_xlabel('$E_a$/eV')
ax[0].set_ylabel('$p(E_a)$')
ax[1].hist(s.preexponential_factor.samples)
ax[1].set_xlabel('$A$/eV')
ax[1].set_ylabel('$p(a)$')
plt.tight_layout()
plt.show()

It is also possible to plot these probability distributions as Arrhenius relations on the data measured values.

In [ ]:
plt.errorbar(1000/s.x, s.y.n, s.y.s, marker='o', ls='', zorder=10)
for i in np.random.randint(s.activation_energy.size, size=128):
    plt.plot(1000/s.x, s.function(s.x, *s.get_sample(i)), color='k', alpha=0.05)
plt.yscale('log')
plt.xlabel('$1000T^{-1}$/K$^{-1}$')
plt.ylabel('$D$/cm$^2$s$^{-1}$')
plt.show()